In [1]:
import shioaji as sj
# from pymongo import MongoClient
from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime
import numpy as np
from Messenger.LineMessenger import LineMessenger as Line
from time import sleep
import pytz

import warnings
warnings.filterwarnings("ignore")

from DataIO import *
from Utils import *

from prettytable import PrettyTable
import json
import os 
parent = os.path.dirname(os.path.abspath("__file__"))
StrongPath = os.path.join(parent, 'StrongTickers')
if not os.path.isdir(StrongPath):
    os.makedirs(StrongPath)
trade_result_path = os.path.join(parent, "TradeRecord")
if not os.path.isdir(trade_result_path):
    os.makedirs(trade_result_path)

In [2]:
class DataObject:
    
    opening = False
    closing = False
    pre_time = "09:00:00"
    name:str=""
    pre_volume:int = 0
    pre_close:int = 0
    total_v:int = 0
    q20_data:dict = {}
    q80_data:dict = {} 
    open_threshold:float = 0
    _api = None
    symbol:str = ""
    refPrice:float = 0
    entry_percent:float = .06
    exit_percent:float = .09
    sl_ratio:float = .03
    short_sl_ratio:float = .02
    v_threshold:float = .01
    entry_threshold:float = 0
    exit_threshold:float = 0
    max_ret:float = 0
    tmp_ret:float = 0
    pos = 0
    traded = 0
    entry = 0
    entry_time = ""
    trade_value = 0
    order = None
    onOrderProcess = False

    preHigh = None
    preLow = None
    sl_threshold = 0
    
    open_l = None
    open_h = None
    
    long_traded = 0
    short_traded = 0
    
    pnl:float = 0
    
    def __init__(self, contract, open_threshold:float, api=None, PreData:dict=None,
                 v_threshold:float=.01, entry_percent:float=.06, exit_percent:float=.09, 
                 takeprofit:float=.6, start_moving_take = .015, sl_ratio:float=.02, max_size:float = 1):
        self.contract = contract
        self.open_threshold = open_threshold
        self.entry_percent = entry_percent
        self.exit_percent = exit_percent
        self.takeprofit = takeprofit
        self.start_moving_take = start_moving_take
        self.sl_ratio = sl_ratio
        self.v_threshold = v_threshold
        self._initialByContract(contract)
        self.max_size = max_size
        self._api = api
        self.first_5mink = {'open':None, 'high':0,'low':9999, 'close':0}
        if PreData:
            self.preHigh = PreData['PreHigh']
            self.preLow = PreData['PreLow']
        
    def _initialByContract(self, contract):
        self.symbol = contract.code
        self.refPrice = contract.reference
        self.entry_threshold = self.refPrice * (1 + self.entry_percent)
        self.exit_threshold = self.refPrice * (1 + self.exit_percent)
        self.name = contract.name
        self.barG = BarGenerator(self.symbol, callback=self.updateSignalWithBar)#, date=date)
        
    def updateQ20Dict(self, data:dict={}):
#         print(data)
        if data is None:return
        if data['simulate'] :return
        if data['symbol'] != self.symbol: return
        self.q20_data = data
        self.q20_data['timeStr'] = self.q20_data['datetime'].strftime("%H:%M:%S")
#         total_volume += qty
        self.updateSignal()
        self.updateStatus()
        try:
            tick = TickData(time=data['datetime'], close=data['close'],volume=data['qty'])
            self.barG.updateBar(tick)
        except Exception as e:
            print(f"update Tick and Bar, Error :{e}")
    
    def updateQ80Dict(self, data:dict):

#         print(data)
        if data['symbol'] != self.symbol: return
        self.q80_data = data
    
    def updateOrderDeal(self, stat, data):
        
        if not (self.closing or self.opening): return 
        signal_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        if stat == sj.constant.OrderState.TFTOrder:
            if data['status']['cancel_quantity'] == data['order']['quantity']:
                self.opening = self.closing = onOrderProcess = False
                return
        elif stat == sj.constant.OrderState.TFTDeal:
            self.pos += (1 if data['action'] == sj.constant.Action.Buy else -1) * data['quantity']
            if self.pos:
                self.entry = data['price']
                print(f"{self.symbol} current pos = {self.pos} with entry price = {self.entry}")
                self.sl_threshold = self.entry * (1 - self.sl_ratio * np.sign(self.pos))
                self.entry_time = signal_time
                self.opening = False
                if self.pos < 0:
                    self.short_sl_ratio = max(self.open_h / self.entry - 1, self.sl_ratio)
                    self.sl_threshold = self.entry * (1 - self.short_sl_ratio * np.sign(self.pos))
            else:
                exit = data['price']
#                 signal_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
                print(f"{self.symbol}, exit price : {exit}")
#                 tmp_pnl = (exit - self.entry) * 1000 * (1 if data['action'] == sj.constant.Action.Sell else -1)
#                 tmp_pnl -= int(get_commission(self.entry if data['action'] == sj.constant.Action.Sell else exit)) # 進場成本
#                 tmp_pnl -= int(get_commission(exit if data['action'] == sj.constant.Action.Sell else self.entry, direction='EXIT', dayTrade=True)) # 出場成本
                tmp_pnl = self.calculatePnL(self.entry, exit, data['action'])
                tmp_ret = tmp_pnl / (self.entry * 1000) # - 1
                self.pnl += tmp_pnl

                cover_text = '\n\n---------------PnL Summary--------------\n'
                cover_text = f'時間 : {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n'
                cover_text += f'Exit Position of {self.symbol} {self.name}\n'
                cover_text += f'Entry : {self.entry} at {self.entry_time.split(".")[0]}\n'
                cover_text += f'Exit : {exit} at {signal_time.split(".")[0]}\n'
                cover_text += f'Partial PnL with Cost : {round(tmp_pnl)}\n'
                cover_text += f'Total PnL with Cost : {round(self.pnl)}\n'
                cover_text += f'Total Ret with Cost : {round(self.tmp_ret * 100, 2)}%\n'
                cover_text += f'Total PnL with Cost of max size "{self.max_size}": {self.max_size * round(self.pnl)}\n'
                cover_text += f'Trade Value: {self.entry * 1000}\n'
                Line.sendMessage(cover_text)
                print('\n---------------PnL Summary--------------')
                print(cover_text)
                if data['action'] == sj.constant.Action.Buy:
                    self.short_traded = 1
                else:
                    self.long_traded = 1
                self.traded = (self.long_traded and self.short_traded) or self.pnl > 0
                self.closing = False
                self.max_ret = self.tmp_ret = 0
                self.entry_time = ""
                self.trade_value += (self.entry if data['action'] == sj.constant.Action.Sell else exit) * 1000
                self.entry = 0
        
    def calculatePnL(self, entry, exit, side=None, simulate=False, pos = 0):
        tmp_pnl = 0
        if pos:
            tmp_pnl += (exit - entry) * 1000 * pos
            tmp_pnl -= int(get_commission(entry if pos > 0 else exit)) # 進場成本
            tmp_pnl -= int(get_commission(exit if pos > 0 else entry, direction='EXIT', dayTrade=True)) # 出場成本
        else:
            if simulate:
                tmp_pnl += (exit - entry) * 1000 * (1 if side == 'S' else -1)
                tmp_pnl -= int(get_commission(entry if side == 'S' else exit)) # 進場成本
                tmp_pnl -= int(get_commission(exit if side == 'S' else entry, direction='EXIT', dayTrade=True)) # 出場成本
            else:
                tmp_pnl += (exit - entry) * 1000 * (1 if side == sj.constant.Action.Sell else -1)
                tmp_pnl -= int(get_commission(entry if side == sj.constant.Action.Sell else exit)) # 進場成本
                tmp_pnl -= int(get_commission(exit if side == sj.constant.Action.Sell else entry, direction='EXIT', dayTrade=True)) # 出場成本
        return tmp_pnl
    
    def updateStatus(self):
        """
        Chech position profit loss
        """
        if not self.pos: return
        if self.traded: return
        if self.closing or self.opening: 
            return
        close = self.q20_data['close']
        tmp_pnl = self.calculatePnL(self.entry, close, pos=np.sign(self.pos), simulate=False)#, simulate=False)
        if self.pos > 0:
            self.tmp_ret = ((close / self.entry) - 1) * np.sign(self.pos)
        else:
            self.tmp_ret = (tmp_pnl / (self.entry * 1000))# - 1
#         self.tmp_ret = ((close / self.entry) - 1) * self.pos
        
        if self.tmp_ret > self.max_ret:
            self.sl_threshold = close * (1 - self.sl_ratio * np.sign(self.pos))
#             self.sl_threshold = self.entry * (1 - self.sl_ratio * self.pos)
            if self.pos < 0:
                self.sl_threshold = close * (1 - self.short_sl_ratio * np.sign(self.pos))
                if self.max_ret >= self.sl_ratio:
                    self.short_sl_ratio = self.sl_ratio
                    self.sl_threshold = close * (1 - self.sl_ratio * np.sign(self.pos))
#                 self.sl_threshold = min(self.sl_threshold, (self.open_h + self.open_l) / 2)
        self.max_ret = max(self.max_ret, self.tmp_ret)
        # print(self.symbol, self.tmp_ret, self.max_ret, close, self.pos)
        
        do_exit_take = False
        if self.pos > 0:
            do_exit_take = (self.tmp_ret >= 0.01)
        elif self.pos < 0:
            if not self.long_traded:
                do_exit_take = (self.tmp_ret >= 0.01)
            try:
                if (self.entry_time.split(' ')[1] >= "12:00:00.000000"):
                    do_exit_take = (self.tmp_ret >= 0.01)
            except:
                if (self.entry_time.split(' ')[0] >= "12:00:00.000000"):
                    do_exit_take = (self.tmp_ret >= 0.01)
#         do_exit_take = (self.tmp_ret >= 0.01) and ((self.entry_time.split(' ')[1] >= "12:00:00.000000") or (self.pos > 0))
#         do_exit_take = self.tmp_ret >= .01
#         if self.max_ret >= self.start_moving_take:
#             do_exit_take = (self.tmp_ret / self.max_ret) <= (self.takeprofit) if self.max_ret else False
        do_exit_stop = close > self.sl_threshold if self.pos == -1 else close < self.sl_threshold
#         print(close > self.sl_threshold if self.pos == -1 else close < self.sl_threshold, (close * self.pos) > (self.sl_threshold * self.pos))
        do_exit = close >= self.exit_threshold if self.pos > 0 else close <= (self.refPrice * (1 - self.exit_percent))
        exit_end = self.q20_data['timeStr'] >= '13:00:00:000000'
        if any([do_exit, do_exit_take, do_exit_stop, exit_end]):
            print(f"{self.symbol} Exit with condition : do_exit : {do_exit}, do_exit_take :{do_exit_take}, do_exit_stop : {do_exit_stop}, exit_end : {exit_end}")
#             print(self.tmp_ret >= 0.01, self.entry_time.split(' ')[1] >= "12:00:00.000000", self.pos == 1)
            self.closing=True
            self.DoTrade('S' if self.pos > 0 else 'B', True)
            exit_string = "停利"
            if do_exit_stop:
                exit_string = "停損"
            if do_exit:
                exit_string = "最佳出場位置"
            if exit_end:
                exit_string = "提前出場"
            self.sendNotifyExit(self.q20_data['timeStr'], self.symbol, self.name,
                             close, self.q20_data['pct_chg'], exit_string)
    
    def updateSignal(self):
        try:
            if not self.q20_data : return
            if self.pos: return
            if self.long_traded or self.traded: return
            totalV = self.q20_data['totalQty']
            close = self.q20_data['close']
            volume = self.q20_data['qty']
            if self.q20_data['timeStr'] != self.pre_time:
                self.total_v = volume
                self.pre_time = self.q20_data['timeStr']
                self.pre_volume = totalV
            else:
                self.total_v += volume
            if not self.pre_close: 
                self.pre_close = close
                if self.first_5mink['open'] is None:
                    self.first_5mink['open'] = close
                self.first_5mink['high'] = max(self.first_5mink['high'], close)
                self.first_5mink['low'] = min(self.first_5mink['low'], close)
                self.first_5mink['close'] = close
                self.pre_volume = totalV
                return

            self.pre_volume = totalV
            signal_type = ""
            if self.q20_data['timeStr'] < '09:05:00:000000':
                self.first_5mink['high'] = max(self.first_5mink['high'], close)
                self.first_5mink['low'] = min(self.first_5mink['low'], close)
                self.first_5mink['close'] = close

            else: 
                if self.q20_data['timeStr'] >= '13:00:00:000000': return
                if all([not self.closing, not self.opening]):
                    sig = 0
                    if close >= max(self.first_5mink['open'], self.first_5mink['close'], self.preHigh if self.preHigh else 0) * 1.005 and close <= self.entry_threshold:
                        print("Entry Long signal with Cross Over")
                        signal_type = "跟上起飛的節奏"
                        sig = 1
                    if sig and not self.opening:
                        self.opening = True
                        self.DoTrade('B' if sig > 0 else 'S', True)
                        self.sendNotify(self.q20_data['datetime'].strftime("%H:%M:%S.%f"), self.symbol, self.name,
                                        close, self.q20_data['pct_chg'], 'B' if sig > 0 else 'S',
                                        signal_type = signal_type)
            self.pre_close = close
        except Exception as e:
            print(f"Update signal of {self.symbol}, Error : {e}")
            
        
    def updateSignalWithBar(self, update_bar):
        try:
            signal_type = ""
            if not self.q20_data : return
            if len(self.barG.bars) < 2:
                return
            if len(self.barG.bars) >= 3 and not (self.open_h or self.open_l):
                self.open_h = max(self.barG.highs[:2])
                self.open_l = min(self.barG.lows[:2])
                print(f"{self.symbol}, open_h : {self.open_h}, open_l : {self.open_l}")
            if not (self.open_h or self.open_l): return
            if self.pos: return
            if self.short_traded or self.traded: return
            if self.q20_data['timeStr'] >= '13:00:00:000000': return
            price = self.q20_data['close']
            sig = 0
#             print(update_bar)
            if self.open_l:
                if price < self.open_l and self.q20_data['timeStr'] >= "12:00:00.000000" and not self.short_traded:
                    if price > self.refPrice * (1 - self.entry_percent):
                        print(f"Entry Short signal with price touch for {self.symbol}")
                        signal_type = "尾盤追殺"
                        sig = -1
                elif update_bar:
                    
#                     print(update_bar['close'], self.open_l, update_bar['low'], self.refPrice)
                    if update_bar['close'] < self.open_l and not self.short_traded: # update_bar['open']
                        if price > self.refPrice * (1 - self.entry_percent):
                            if self.open_h >=  self.refPrice * (1 + self.exit_percent):return
                            print(f"5m Bar CLose :{update_bar['close']}, open low : {self.open_l}")
                            print("Entry Short signal with price low open threshold")
                            signal_type = "破開盤底等跳水"
                            sig = -1
                    elif update_bar['close'] > self.open_l and update_bar['low'] < self.open_l and not self.long_traded:
                        if price < self.refPrice * (1 + self.entry_percent):
                            print("Entry Long signal with V Turn")
                            signal_type = "V轉做多"
                            sig = 1
            if sig and not self.opening:
                self.opening = True
                self.DoTrade('B' if sig > 0 else 'S', True)
                self.sendNotify(self.q20_data['timeStr'], self.symbol, self.name,
                                price, self.q20_data['pct_chg'], 'B' if sig > 0 else 'S', 
                                signal_type = signal_type)
        except Exception as e:
            print(f"Update signal with bar of {self.symbol}, Error : {e}")
        
        
    def DoTrade(self, side, simulate=False):
        try:
            close = self.q20_data['close']
            #=========
            # RealTime
            #=========
            if not simulate:
                if not self._api:return
                order_price = self.q80_data['ask1'] if side == 'B' else self.q80_data['bid1']
                print(f"{self.symbol}, close : {close}, entry price : {order_price}")
                if abs(order_price / close - 1) >= .005:
                    order_price = close
                order = self._api.Order(
                    price=order_price,
                    quantity=1,
                    action=sj.constant.Action.Buy if side == 'B' else sj.constant.Action.Sell,
                    price_type=sj.constant.StockPriceType.LMT,
                    order_type=sj.constant.TFTOrderType.ROD,
                    first_sell=sj.constant.StockFirstSell.Yes,# if side == 'S' else sj.constant.StockFirstSell.No,
                )
                self.order = self._api.place_order(self.contract, order)
                if self.order.status.status in [sj.constant.Status.PendingSubmit, sj.constant.Status.Submitted]:
                    if not self.pos:
                        self.opening = True
                    else:
                        self.closing = True
            #=========
            # Simulate
            #=========
            else:
                if not any([self.opening, self.closing]):return
#                 if self.pos: return
                self.pos += 1 if side == 'B' else -1
                signal_time = self.q20_data["timeStr"]#.strftime("%H:%M:%S.%f")
                if self.pos == 0:
                    self.traded = True
                    exit = self.q80_data['ask1'] if side == 'B' else self.q80_data['bid1']
                    print(f"{self.symbol}, close : {close}, exit price : {exit}")
                    if abs(exit / close - 1) >= .005:
                        exit = close
                    tmp_pnl = (exit - self.entry) * 1000 * (1 if side == 'S' else -1)
                    tmp_pnl -= int(get_commission(self.entry if side == 'S' else exit)) # 進場成本
                    tmp_pnl -= int(get_commission(exit if side == 'S' else self.entry, direction='EXIT', dayTrade=True)) # 出場成本
                    tmp_ret = tmp_pnl / (self.entry * 1000) # - 1
                    self.pnl += tmp_pnl
                    self.trade_value += self.entry * 1000


                    cover_text = '\n\n---------------PnL Summary--------------\n'
                    cover_text = f'時間 : {datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")}\n'
                    cover_text += f'Exit Position of {self.symbol} {self.name}\n'
                    cover_text += f'Entry : {self.entry} at {self.entry_time}\n'
                    cover_text += f'Exit : {exit} at {signal_time}\n'
                    cover_text += f'Total PnL with Cost : {round(tmp_pnl)}\n'
                    cover_text += f'Total Ret with Cost : {round(tmp_ret * 100, 2)}%\n'
                    cover_text += f'Total PnL with Cost of max size "{self.max_size}": {self.max_size * round(tmp_pnl)}\n'
                    cover_text += f'Trade Value: {self.entry * 1000}\n'
                    Line.sendMessage(cover_text)
#                     print('\n---------------PnL Summary--------------')
                    print(cover_text)

    #                 cover_text = '\n\n---------------PnL Summary--------------\n'
    #                 cover_text = f'時間 : {datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")}\n'
    #                 cover_text += f'Exit Position of {self.symbol}\n'
    #                 cover_text += f'Entry : {self.entry} at {self.entry_time}\n'
    #                 cover_text += f'Exit : {exit} at {signal_time}\n'
    #                 cover_text += f'Partial PnL with Cost : {round(tmp_pnl)}\n'
    #                 cover_text += f'Total PnL with Cost : {round(tmp_pnl)}\n'
    #                 cover_text += f'Total Ret with Cost : {round(tmp_ret * 100, 2)}%\n'
    #                 cover_text += f'Total PnL with Cost of max size "{self.max_size}": {self.max_size * round(tmp_pnl)}\n'
    #                 cover_text += f'Trade Value: {round(self.entry * 1000)}\n'
    #                 Line.sendMessage(cover_text)
    #                 print('\n---------------PnL Summary--------------')
    #                 print(f'Exit Position of {self.contract.code}')
    #                 print(f'Entry : {self.entry} at {self.entry_time}')
    #                 print(f'Exit : {exit} at {signal_time}')
    #                 print(f'Total PnL with Cost : {round(tmp_pnl)}')
    #                 print(f'Total Ret with Cost : {round(tmp_ret * 100, 2)}%')
    #                 print(f'Total PnL with Cost of max size {self.max_size}: {self.max_size * round(tmp_pnl)}\n')

                    self.entry = 0
                    self.closing = False
                    return
                self.entry = self.q80_data['ask1'] if side == 'B' else self.q80_data['bid1']
                print(f"{self.symbol}, close : {close}, entry price : {self.entry}")
                if abs(self.entry / close - 1) >= .005:
                    self.entry = close
                self.sl_threshold = self.entry * (1 - self.sl_ratio)
                self.entry_time = signal_time
                self.opening = False
        except Exception as e:
            print(f"Trade of {self.symbol}, Error : {e}")
    def sendNotify(self, dateStr, idx, name, 
                   close, Ret, Side="B", signal_type=""):#, TVRatio, EVRatio):
        try:
            text = f'時間 : {dateStr}\n'
            text += f'股票代號/名稱 : {idx}/{name}\n'
            if Side == "B":
                text += f'觸發條件 : 突破第一個5分K高點\n'
                text += f'進場方向 : 做多\n'
            else:
                text += f'觸發條件 : 突破第一個5分K低點\n'
                text += f'進場方向 : 做空\n'
            text += f'成交價 : {close}\n'
            
            
            # 漲跌幅量
            text += f'漲跌幅 : {Ret} %\n'
            # text += f'量比(總/估) {TVRatio}/{EVRatio}\n'
            # text += '其他提醒 : \n'
            text += f'訊號類型 : {signal_type}\n'
            
            Line.sendMessage(text)
        except Exception as e:
            print(e)

    def sendNotifyExit(self, dateStr, idx, name, 
                   close, Ret, exit_string=""):
        try:
            text = f'時間 : {dateStr}\n'
            text += f'股票代號/名稱 : {idx}/{name}\n'
            text += f'觸發出場條件 : \n\t已達到( {exit_string} )條件囉\n'
            text += f'成交價 : {close}\n'
            
            # 漲跌幅量
            text += f'漲跌幅 : {Ret} %\n'
            # text += f'量比(總/估) {TVRatio}/{EVRatio}\n'
            # text += '其他提醒 : \n'
            
            Line.sendMessage(text)
        except Exception as e:
            print('send Exit, Error : ' + e)

In [3]:
trade_dt = datetime.today() # datetime(2022,1,11) # 
# trade_dt = datetime(2022,5,11) # 
trade_dtStr = trade_dt.strftime("%Y-%m-%d")
open_thresholds = readStrongTicker(trade_dtStr)

In [4]:
api = sj.Shioaji()

api.login("F128497445", "89118217k")
api.activate_ca(os.path.join(parent, 'Sinopac.pfx'), 'j7629864', 'F128497445')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


True

In [5]:
from shioaji import TickSTKv1, Exchange, BidAskSTKv1, TickFOPv1, BidAskFOPv1

def q20_callback(exchange:Exchange, tick:[TickSTKv1, TickFOPv1]):
    try:
        NotifyTickers[tick.code] = NotifyTickers.get(tick.code,
                                                DataObject(tmp_contract, open_threshold=tmp_contract.reference, 
                                                api=api, max_size = max_size_map[tick.code], PreData=open_thresholds[tick.code]))

        NotifyTickers[tick.code].updateQ20Dict(dict(
            symbol=tick.code,
            datetime=tick.datetime,
            open=float(tick.open),
            high=float(tick.high),
            low=float(tick.low),
            close=float(tick.close),
            avg_price=float(tick.avg_price),
            qty=int(tick.volume),
            totalQty=int(tick.total_volume),
            pct_chg=float(tick.pct_chg),
            simulate=bool(tick.simtrade),
        ))
    except Exception as e:
        print(e)

def q80_callback(exchange:Exchange, tick:[BidAskSTKv1, BidAskFOPv1]):
    try:
        NotifyTickers[tick.code] = NotifyTickers.get(tick.code,
                                                DataObject(tmp_contract, open_threshold=tmp_contract.reference, 
                                                api=api, max_size = max_size_map[tick.code], PreData=open_thresholds[tick.code]))

        NotifyTickers[tick.code].updateQ80Dict(dict(
            symbol = tick.code,
            datetime = tick.datetime,
            bid1 = float(tick.bid_price[0]),
            bid2 = float(tick.bid_price[1]),
            bid3 = float(tick.bid_price[2]),
            bid4 = float(tick.bid_price[3]),
            bid5 = float(tick.bid_price[4]),
            bidQty1 = float(tick.bid_volume[0]),
            bidQty2 = float(tick.bid_volume[1]),
            bidQty3 = float(tick.bid_volume[2]),
            bidQty4 = float(tick.bid_volume[3]),
            bidQty5 = float(tick.bid_volume[4]),
            askQty1 = float(tick.ask_volume[0]),
            askQty2 = float(tick.ask_volume[1]),
            askQty3 = float(tick.ask_volume[2]),
            askQty4 = float(tick.ask_volume[3]),
            askQty5 = float(tick.ask_volume[4]),
            ask1 = float(tick.ask_price[0]),
            ask2 = float(tick.ask_price[1]),
            ask3 = float(tick.ask_price[2]),
            ask4 = float(tick.ask_price[3]),
            ask5 = float(tick.ask_price[4]),
        ))
    except Exception as e:
        print(e)
    
api.quote.set_on_tick_stk_v1_callback(q20_callback)
api.quote.set_on_bidask_stk_v1_callback(q80_callback)
api.quote.set_on_bidask_fop_v1_callback(q80_callback)
api.quote.set_on_tick_fop_v1_callback(q20_callback)

def place_cb(stat, msg):
    try:
        if stat == sj.constant.OrderState.TFTOrder:
            if msg['order']['order_cond'] == 'Cash':
                if msg['contract']['code'] in NotifyTickers.keys():
                    NotifyTickers[msg['contract']['code']].updateOrderDeal(stat, msg)
        else:
            if msg['order_cond'] == 'Cash':
                if msg['code'] in NotifyTickers.keys():
                    NotifyTickers[msg['code']].updateOrderDeal(stat, msg)
    except:
        pass
    print(f"Order Status : {stat}, Order Data : {msg} \n")
api.set_order_callback(place_cb)

In [6]:
# remove can't daytrade ticker
tmp_ticker_list = list(open_thresholds.keys())
for ticker in tmp_ticker_list:
#     print(ticker)
    tmp_contract = api.Contracts.Stocks[str(ticker)]
#     print(tmp_contract, tmp_contract.day_trade)
    if tmp_contract.day_trade == sj.constant.DayTrade.Yes:continue
    open_thresholds.pop(ticker)
    
writeTradableTicker(trade_dtStr, open_thresholds)
    
# seperate capital for each ticker
total_capital = 5e6
seperated_capital = int(total_capital / len(open_thresholds.keys()))
max_size_map = {}
for k, v in open_thresholds.items():
    max_pos = int(seperated_capital / (v['PreClose'] * 1000))
    print(f'{k} 最大倉位 : {max_pos}, 昨收 : {v}')
    max_size_map[k] = max_pos

1236 最大倉位 : 23, 昨收 : {'PreClose': 29.95, 'PreHigh': 29.95, 'PreLow': 27.05}
1539 最大倉位 : 24, 昨收 : {'PreClose': 29.35, 'PreHigh': 29.35, 'PreLow': 27.05}
1783 最大倉位 : 14, 昨收 : {'PreClose': 48.3, 'PreHigh': 48.3, 'PreLow': 43.05}
6206 最大倉位 : 7, 昨收 : {'PreClose': 93.4, 'PreHigh': 94.0, 'PreLow': 85.2}
6140 最大倉位 : 34, 昨收 : {'PreClose': 20.7, 'PreHigh': 20.7, 'PreLow': 18.0}
6461 最大倉位 : 24, 昨收 : {'PreClose': 29.55, 'PreHigh': 29.55, 'PreLow': 27.05}
6472 最大倉位 : 2, 昨收 : {'PreClose': 261.5, 'PreHigh': 261.5, 'PreLow': 240.5}


In [7]:
tradable_text_table = createTradableTable(open_thresholds, max_size_map)
text = f'\n【{trade_dtStr}】當沖策略標的\n\n'
text += tradable_text_table.get_string()
Line.sendMessage(text)

200

In [8]:
NotifyTickers = {}
for i in range(125):
    if i < len(open_thresholds.keys()):
        ticker = sorted(open_thresholds.keys())[i]
        if open_thresholds[ticker]['PreClose']>1000:continue
        tmp_contract = api.Contracts.Stocks[ticker]
        if tmp_contract.day_trade  != sj.constant.DayTrade.No:
            print("Tradable Ticker : " + ticker)
            # tmp_contract = api1.Contracts.Futures[ticker]
#             NotifyTickers[ticker] = NotifyTickers.get(ticker,
#                                                       DataObject(tmp_contract, open_threshold=tmp_contract.reference, 
#                                                                 api=api, max_size = max_size_map[ticker], PreData=open_thresholds[ticker]))
            NotifyTickers[ticker] = DataObject(tmp_contract, open_threshold=tmp_contract.reference, 
                                                                api=api, max_size = max_size_map[ticker], PreData=open_thresholds[ticker])
            
            api.quote.subscribe(
                tmp_contract, 
                quote_type = sj.constant.QuoteType.Tick, # or 'tick'
                version = sj.constant.QuoteVersion.v1 # or 'v1'
            )
            api.quote.subscribe(
                tmp_contract, 
                quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
                version = sj.constant.QuoteVersion.v1 # or 'v1'
            )

Tradable Ticker : 1236
Tradable Ticker : 1539
Tradable Ticker : 1783
Tradable Ticker : 6140
Tradable Ticker : 6206
Tradable Ticker : 6461
Tradable Ticker : 6472
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1236 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1236 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1539 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1539 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1783 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1783 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/OTC/6140 | Event: Subscribe or Unsubscribe ok

In [9]:
NotifyTickers['4503'].q20_data

KeyError: '4503'

In [10]:
while 1:
    try:
        sleep(1)
    except:
        break

Order Status : TFTORDER, Order Data : {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': 'a5af02f7', 'seqno': '194644', 'ordno': 'IB087', 'account': {'account_type': 'S', 'person_id': '', 'broker_id': '9A92', 'account_id': '0231901', 'signed': True}, 'action': 'Sell', 'price': 50.8, 'quantity': 2, 'order_type': 'ROD', 'price_type': 'LMT', 'order_cond': 'MarginTrading', 'order_lot': 'Common', 'custom_field': ''}, 'status': {'id': 'a5af02f7', 'exchange_ts': 1656550817, 'modified_price': 0.0, 'cancel_quantity': 0, 'order_quantity': 2, 'web_id': '148'}, 'contract': {'security_type': 'STK', 'exchange': 'TSE', 'code': '2408', 'symbol': '', 'name': '', 'currency': 'TWD'}} 

Order Status : TFTDEAL, Order Data : {'trade_id': 'a5af02f7', 'seqno': '194644', 'ordno': 'IB087', 'exchange_seq': '032296', 'broker_id': '9A92', 'account_id': '0231901', 'action': 'Sell', 'code': '2408', 'order_cond': 'MarginTrading', 'order_lot': 'Common', 'price': 50.9, 'quantity': 1, 'web_

In [11]:
for ticker in NotifyTickers.keys():
    api.quote.unsubscribe(
        NotifyTickers[ticker].contract, 
        quote_type = sj.constant.QuoteType.Tick, # or 'tick'
        version = sj.constant.QuoteVersion.v1 # or 'v1'
    )
    api.quote.unsubscribe(
        NotifyTickers[ticker].contract, 
        quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
        version = sj.constant.QuoteVersion.v1 # or 'v1'
    )

Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1236 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1236 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1539 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1539 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1783 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1783 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/OTC/6140 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/OTC/6140 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/6206 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/6206 | Event

In [12]:
single_pnl = 0
max_pnl = 0
ticker_list = []
pnl_list = []
max_pnl_list = []
max_size_list = []
ret_list = []
est_total_value = 0
est_single_value = 0
for d in NotifyTickers.values():
    print(d.__dict__, '\n')
    if d.long_traded or d.short_traded or d.traded:
        ticker_list.append(d.symbol)
        pnl_list.append(int(d.pnl))
        max_size_list.append(d.max_size)
        max_pnl_list.append(int(d.pnl * d.max_size))
        ret_list.append(str(round(d.tmp_ret * 100, 2)) + "%")
        est_single_value += d.trade_value # d.open_threshold * 1000
        est_total_value += d.trade_value * int(d.max_size / 2) # d.open_threshold * 1000 * d.max_size
        # print(d.symbol, int(d.pnl), str(round(d.tmp_ret * 100, 2)) + "%", d.max_size, int(d.pnl * d.max_size))
        single_pnl += d.pnl
        max_pnl += d.pnl * d.max_size

ticker_list.append("合計")
pnl_list.append(int(single_pnl))
max_size_list.append(est_total_value)
max_pnl_list.append(int(max_pnl))
ret_list.append((str(round((max_pnl / est_total_value) * 100, 2)) + "%") if est_total_value else "0%")

{'contract': Stock(exchange=<Exchange.TSE: 'TSE'>, code='1236', symbol='TSE1236', name='宏亞', category='02', unit=1000, limit_up=32.9, limit_down=27.0, reference=29.95, update_date='2022/06/30', margin_trading_balance=49, short_selling_balance=25, day_trade=<DayTrade.Yes: 'Yes'>), 'open_threshold': 29.95, 'entry_percent': 0.06, 'exit_percent': 0.09, 'takeprofit': 0.6, 'start_moving_take': 0.015, 'sl_ratio': 0.02, 'v_threshold': 0.01, 'symbol': '1236', 'refPrice': 29.95, 'entry_threshold': 31.747, 'exit_threshold': 32.6455, 'name': '宏亞', 'barG': <Utils.BarGenerator object at 0x122883c10>, 'max_size': 23, '_api': <shioaji.shioaji.Shioaji object at 0x120438d90>, 'first_5mink': {'open': 29.95, 'high': 30.8, 'low': 29.4, 'close': 30.7}, 'preHigh': 29.95, 'preLow': 27.05, 'q80_data': {'symbol': '1236', 'datetime': datetime.datetime(2022, 6, 30, 14, 30), 'bid1': 28.95, 'bid2': 28.9, 'bid3': 28.85, 'bid4': 28.8, 'bid5': 28.75, 'bidQty1': 14.0, 'bidQty2': 1.0, 'bidQty3': 12.0, 'bidQty4': 2.0, 'b

In [13]:

def createSummaryTable(tickers, pnl_list:list=None, max_pnl_list:list=None, max_size_list:list=None, ret_list:list=None):
    try:
        table = PrettyTable()
        table.add_column('代號', tickers)
        table.add_column("損益", pnl_list)
        table.add_column("報酬率", ret_list)
        table.add_column("最大倉位", max_size_list)
        table.add_column("最大倉損益", max_pnl_list)
        table.align = 'r'
    except Exception as e:
        print(e)
    else:
        return table
    
sum_table = createSummaryTable(ticker_list, pnl_list, max_pnl_list, max_size_list, ret_list)

In [14]:
text = f'\n【{trade_dtStr}】策略損益\n'
text += f'總損益(單部位) : {int(single_pnl)}\n'
text += f'成交量(單部位) : {est_single_value}\n'
text += f'總報酬(單部位) : {str(round((int(single_pnl) / est_single_value) * 100, 2)) if est_single_value else "0"}%\n\n'

text += f'總損益(最大倉) : {int(max_pnl)}\n'
text += f'成交量(最大倉) : {est_total_value}\n'
text += f'總報酬(最大倉) : {str(round((int(max_pnl) / est_total_value) * 100, 2)) if est_total_value else "0"}%\n'
text.split('\n')#.replace('\n', '<br>')

['',
 '【2022-06-30】策略損益',
 '總損益(單部位) : -6481',
 '成交量(單部位) : 528900.0',
 '總報酬(單部位) : -1.23%',
 '',
 '總損益(最大倉) : -55028',
 '成交量(最大倉) : 2329400.0',
 '總報酬(最大倉) : -2.36%',
 '']

In [15]:
with open(os.path.join(trade_result_path,f'{trade_dtStr}_ResultTable.html'), 'w') as f:
    f.write(sum_table.get_html_string())

In [17]:
Line.sendMessage(text)

200

In [16]:
def CreateTradeRecord(api, trade_dtStr, dayTrade = True):
    col_names = {
        'date':"交易日期",
        'cond':'委託種類',
        'code':"商品代碼",
        'quantity':"數量",
        'price':'進場價格',
        'cost':'進場成本',
        'dseq':'委託單號',
        'rep_margintrading_amt':"融資金額",
        'rep_collateral':"擔保品",
        'rep_margin':'保證金',
        'fee':'進場手續費',
        'interest':'利息',
        'tax':'交易稅',
        'shortselling_fee':"借券費",
        'currency':"幣別",
        'trade_type':"交易別",
        'exit_price':"出場價格",
        "pnl":"淨損益",
        "pr_ratio":"報酬率"
    }
#     if dayTrade:
#         col_names.update({"pr_ratio":"報酬率"})
    #=====================
    # 從 API 讀取交易紀錄
    #=====================
    pnls_sum = api.list_profit_loss(api.stock_account, trade_dtStr, trade_dtStr)
    
    details = []
    for pnl in pnls_sum:
        for detail in api.list_profit_loss_detail(api.stock_account, pnl['id']):
            d = detail.dict()
            
            if d['trade_type'] == "DayTrade" and dayTrade:
                d.update({
                    'exit_price':pnl['price'],
                    'pnl':pnl['pnl'],
                    'pr_ratio':pnl['pr_ratio']
                })
                details.append(d)
            elif d['trade_type'] != "DayTrade" and not dayTrade:
                d.update({
                    'exit_price':pnl['price'],
                    'pnl':pnl['pnl'],
                    'pr_ratio':pnl['pr_ratio']
                })
                details.append(d)
            
    #=====================
    # 計算交易明細
    #=====================            
    df_Record = DataFrame(details)
    if not df_Record.empty:
#         print(df_Record)
        df_Record = df_Record.rename(columns=col_names)
        df_Record['報酬率'] = (df_Record['報酬率'] * 100).apply(lambda x: str(round(x,2)))+"%"

        df_Record['獲利'] = (df_Record['出場價格'] - df_Record['進場價格']) * 1000 * df_Record['數量']
        df_Record['交易稅'] = (df_Record['出場價格'] * df_Record['數量'] * 3 * (.5 if dayTrade else 1)).fillna(0).apply(lambda x: int(x))
        df_Record['出場手續費'] = round(df_Record['出場價格'] * 1.425)
        df_Record['總交易成本'] = df_Record['交易稅'] + df_Record['進場手續費'] + df_Record['出場手續費']
        if not dayTrade:
            df_Record['總交易成本'] += df_Record['利息']
        
        df_Record['總損益(自結)'] = df_Record['獲利'] - df_Record['總交易成本']
    
    #=====================
    # 計算總和
    #=====================
    summary = {
        '交易日期':"合計",
        '委託種類':"",
        "商品代碼":"",
        "數量":0,
        '進場價格':"",
        '進場成本':0,
        '委託單號':"",
        "融資金額":0,
        "擔保品":0,
        '保證金':0,
        '進場手續費':0,
        '利息':0,
        '交易稅':0,
        "借券費":0,
        "幣別":"NTD",
        "交易別":"",
        "出場價格":"",
        "淨損益":0,
        "報酬率":"0%",
        "獲利":0,
        "出場手續費":0,
        "總交易成本":0,
        "總損益(自結)":0,

    }
    if not df_Record.empty:
        summary.update({
            "數量":int(df_Record['數量'].sum()),
            '進場成本':int((df_Record['進場價格'] * df_Record["數量"] * 1000).sum()),
            "融資金額":int(df_Record['融資金額'].sum()),
            "擔保品":int(df_Record['擔保品'].sum()),
            '保證金':int(df_Record['保證金'].sum()),
            '進場手續費':int(df_Record['進場手續費'].sum()),
            '利息':int(df_Record['利息'].sum()),
            '交易稅':int(df_Record['交易稅'].sum()),
            "借券費":int(df_Record['借券費'].sum()),
            "淨損益":int(df_Record['淨損益'].sum()),
            "獲利":int(df_Record['獲利'].sum()),
            "出場手續費":int(df_Record['出場手續費'].sum()),
            "總交易成本":int(df_Record['總交易成本'].sum()),
            "總損益(自結)":int(df_Record['總損益(自結)'].sum()),
        })
        summary['報酬率'] = f"{round(summary['總損益(自結)'] / summary['進場成本'] * 100, 2)}%"
    summary_df = DataFrame([summary])
    
    #=====================
    # 計算總和
    #=====================
    final_Record = summary_df
    if not df_Record.empty:
        final_Record = df_Record.append(summary_df)
    final_Record = final_Record["交易日期,委託種類,交易別,商品代碼,委託單號,數量,進場價格,進場成本,出場價格,融資金額,擔保品,保證金,利息,借券費,進場手續費,出場手續費,交易稅,獲利,總交易成本,淨損益,總損益(自結),報酬率".split(',')]
    extra_file_name = '當沖' if dayTrade else '隔日沖'
    file_name = os.path.join(trade_result_path, f"{trade_dtStr}_TradingRecord_{extra_file_name}.csv")
#     print(file_name)
    final_Record.to_csv(file_name, encoding='utf-8-sig',index=False)

In [17]:
## temp_str = "2022-03-"
CreateTradeRecord(api, trade_dtStr)
CreateTradeRecord(api, trade_dtStr, False)

# Cross Day Trade

In [32]:
def EntryCrossDayTrade(api, ticker, qty=1, Fixing=False):
    tmp_contract = api.Contracts.Stocks[ticker]
    snap = api.snapshots([tmp_contract])[0]

    order_price = tmp_contract.limit_up
#     if snap.buy_volume < snap.sell_volume:
#         order_price = snap.buy_price
#     else:
#         order_price = snap.sell_price
    orders = []
    if tmp_contract.margin_trading_balance <= 0:# and order_price > 100:
        print(ticker, tmp_contract['name'], '無法進行資買')
        return
    for i in range(qty):
        order = api.Order(
                price=order_price,
                quantity=1,
                action=sj.constant.Action.Buy,
                price_type=sj.constant.StockPriceType.LMT,
                order_type=sj.constant.TFTOrderType.ROD,
                first_sell=sj.constant.StockFirstSell.Yes,# if side == 'S' else sj.constant.StockFirstSell.No,
                order_cond=sj.constant.StockOrderCond.MarginTrading if tmp_contract.margin_trading_balance > 0 else sj.constant.StockOrderCond.Cash,
                order_lot=sj.constant.TFTStockOrderLot.Common if not Fixing else sj.constant.TFTStockOrderLot.Fixing,
            )
        tmp_order = api.place_order(tmp_contract, order)
        if tmp_order.status.status == "Failed":
            print(ticker, tmp_order, '\n')
        orders.append(tmp_order)
    return orders

def ExitCrossDayTrade(api, ticker, qty=1, Fixing=False):
    tmp_contract = api.Contracts.Stocks[ticker]
    snap = api.snapshots([tmp_contract])[0]

    order_price = snap.close
    if snap.buy_volume < snap.sell_volume:
        order_price = snap.buy_price
    else:
        order_price = snap.sell_price
    orders = []
    for i in range(qty):
        order = api.Order(
                price=order_price,
                quantity=1,
                action=sj.constant.Action.Sell,
                price_type=sj.constant.StockPriceType.LMT,
                order_type=sj.constant.TFTOrderType.ROD,
                first_sell=sj.constant.StockFirstSell.Yes,# if side == 'S' else sj.constant.StockFirstSell.No,
                order_cond=sj.constant.StockOrderCond.MarginTrading if tmp_contract.margin_trading_balance > 0 else sj.constant.StockOrderCond.Cash,
                order_lot=sj.constant.TFTStockOrderLot.Common if not Fixing else sj.constant.TFTStockOrderLot.Fixing,
            )
        tmp_order = api.place_order(tmp_contract, order)
        if tmp_order.status.status == "Failed":
            print(ticker, tmp_order, '\n')
        orders.append(tmp_order)
    return orders

In [33]:
CrossDayOrders = {}

In [31]:
# api.logout()
api = sj.Shioaji()

api.login("F128497445", "89118217k")
api.activate_ca(os.path.join(parent, 'Sinopac.pfx'), 'j7629864', 'F128497445')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


True

## Entry

In [35]:
for ticker in '2427,3038,3047'.split(','):
    if len(ticker) > 0:
        CrossDayOrders[ticker] = EntryCrossDayTrade(api, ticker, 1) # 盤中
#         CrossDayOrders[ticker] = EntryCrossDayTrade(api, ticker, 1, True) # 盤後定價

OrderState.TFTOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': 'e3124cc0', 'seqno': '066723', 'ordno': 'IG191', 'account': {'account_type': 'S', 'person_id': '', 'broker_id': '9A92', 'account_id': '0231901', 'signed': True}, 'action': 'Buy', 'price': 14.85, 'quantity': 1, 'order_type': 'ROD', 'price_type': 'LMT', 'order_cond': 'MarginTrading', 'order_lot': 'Common', 'custom_field': ''}, 'status': {'id': 'e3124cc0', 'exchange_ts': 1656394017, 'modified_price': 0.0, 'cancel_quantity': 0, 'order_quantity': 1, 'web_id': '137'}, 'contract': {'security_type': 'STK', 'exchange': 'TSE', 'code': '2427', 'symbol': '', 'name': '', 'currency': 'TWD'}}
OrderState.TFTOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '487838f4', 'seqno': '066724', 'ordno': 'IG192', 'account': {'account_type': 'S', 'person_id': '', 'broker_id': '9A92', 'account_id': '0231901', 'signed': True}, 'action': 'Buy', 'price': 22.85, 'quantity': 1, 'or

In [29]:
api.update_status()

In [30]:
api.list_trades()

[Trade(contract=Stock(exchange=<Exchange.OTC: 'OTC'>, code='4541', symbol='OTC4541', name='晟田', category='20', unit=1000, limit_up=30.9, limit_down=25.3, reference=28.1, update_date='2022/06/20', margin_trading_balance=173, day_trade=<DayTrade.Yes: 'Yes'>), order=Order(action=<Action.Buy: 'Buy'>, price=30.9, quantity=1, id='e8a775ba', seqno='218157', ordno='II643', account=Account(account_type=<AccountType.Stock: 'S'>, person_id='F128497445', broker_id='9A92', account_id='0231901', signed=True), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>, order_cond=<StockOrderCond.MarginTrading: 'MarginTrading'>, first_sell=<StockFirstSell.Yes: 'true'>), status=OrderStatus(id='e8a775ba', status=<Status.Filled: 'Filled'>, status_code='00', web_id='137', order_datetime=datetime.datetime(2022, 6, 20, 13, 27, 45), deal_quantity=1, deals=[])),
 Trade(contract=Stock(exchange=<Exchange.OTC: 'OTC'>, code='8421', symbol='OTC8421', name='旭源', category='20', unit=1000, limit

## Exit

In [27]:
for ticker in ''.split(','):
    if len(ticker) > 0:
#         CrossDayOrders[ticker] = ExitCrossDayTrade(api, ticker, 1) # 盤中
        CrossDayOrders[ticker] = ExitCrossDayTrade(api, ticker, 1, True) # 盤後定價

Order Status : TFTORDER, Order Data : {'operation': {'op_type': 'New', 'op_code': '88', 'op_msg': '融資賣出餘股數不足，餘股數 0 股（自辦）'}, 'order': {'id': 'b11f544b', 'seqno': '081598', 'ordno': 'I0000', 'account': {'account_type': 'S', 'person_id': '', 'broker_id': '9A92', 'account_id': '0231901', 'signed': True}, 'action': 'Sell', 'price': 58.9, 'quantity': 1, 'order_type': 'ROD', 'price_type': 'LMT', 'order_cond': 'MarginTrading', 'order_lot': 'Fixing', 'custom_field': ''}, 'status': {'id': 'b11f544b', 'exchange_ts': 1650261720, 'modified_price': 0.0, 'cancel_quantity': 0, 'order_quantity': 0, 'web_id': '137'}, 'contract': {'security_type': 'STK', 'exchange': 'TSE', 'code': '6217', 'symbol': '', 'name': '', 'currency': 'TWD'}} 

Order Status : TFTORDER, Order Data : {'operation': {'op_type': 'New', 'op_code': '88', 'op_msg': '融資賣出餘股數不足，餘股數 0 股（自辦）'}, 'order': {'id': 'f8f9b234', 'seqno': '081600', 'ordno': 'I0000', 'account': {'account_type': 'S', 'person_id': '', 'broker_id': '9A92', 'account_id':